# 目录
1. 提出问题（Business Understanding ）
2. 理解数据（Data Understanding）
 * 采集数据
 * 导入数据
 * 查看数据集信息
3. 数据清洗（Data Preparation ）
 * 数据预处理
 * 特征工程（Feature Engineering）
4. 构建模型（Modeling） 
5. 模型评估（Evaluation） 
6. 方案实施 （Deployment）
 * 提交结果到Kaggle

1.提出问题

[Titanic](https://www.kaggle.com/c/titanic)

2.导入数据

In [1]:
import numpy as np
import pandas as pd

#训练数据集
train = pd.read_csv("train.csv")
#测试数据集
test = pd.read_csv("test.csv")
#这里要记住训练数据集有891条数据，方便后面从中拆分出测试数据集用于提交Kaggle结果
print('训练数据集:',train.shape,'测试数据集:',test.shape)

训练数据集: (891, 12) 测试数据集: (418, 11)


In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


可以看出发现train比test多出了Survived列，这是生存情况，也就是test需要预测的结果。

In [4]:
#因为要同步清洗训练数据和测试数据，因此将两个数据集合并起来便于操作。
full = train.append(test, ignore_index=True)
print(full.shape)
full.head()

(1309, 12)


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


这里标出了每个column的意义。

In [5]:
full.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


In [6]:
full.info()
#通过full.info()看出，Age Cabin Embarked Fare四列有缺失数据，Cabin列的缺失数据达到了1-295/1309=77.5%。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


3.数据清洗

数据预处理

In [7]:
#1.选择子集
#此题不需要

#2.列名重命名
#此题不需要

#3.缺失数据处理
#数据类型为float的用平均值填充。
full['Age'] = full['Age'].fillna(full['Age'].mean())
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())
#只有2条缺失数据的Embarked列用出现最多的登船港口填充。
Embarked_Rank = full.groupby('Embarked').count().sort_values(by='Name', ascending=False).index.values.tolist()
full['Embarked'] = full['Embarked'].fillna(Embarked_Rank[0])
#Cabin列缺失数据太多，填充为U，表示Unknown
full['Cabin'] = full['Cabin'].fillna('U')

#4.数据类型转换
#此题不需要

#5.数据排序
#此题不需要

#6.异常值处理
#此题不需要

In [8]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [9]:
#特征提取
#将性别映射为数值类型
sex_mapDict = {'male':1, 'female':0}
full['Sex'] = full['Sex'].map(sex_mapDict)
full.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,0,1,1.0,113803
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,1,0,0.0,373450


In [10]:
#将登船港口、客舱等级进行One-hot编码
#存放提取后的登船港口特征
embarkedDf = pd.DataFrame()
#使用get_dummies进行one-hot编码，产生虚拟变量（dummy variables），列名前缀是Embarked
embarkedDf = pd.get_dummies( full['Embarked'] , prefix='Embarked' )
embarkedDf.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [11]:
#存放提取后的客舱等级特征
pclassDf = pd.DataFrame()
#使用get_dummies进行one-hot编码，列名前缀是Pclass
pclassDf = pd.get_dummies(full['Pclass'] , prefix='Pclass')
pclassDf.head()

,Pclass_1,Pclass_2,Pclass_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1


In [12]:
#姓名列包括三部分：名、头衔、姓，头衔是表明身份地位的特征，可能对生存结果有影响，需要提取出来
#存放提取后的头衔特征
titleDf = pd.DataFrame()
import re

titleDf['Title'] = full['Name'].apply(lambda x: re.search(r'(?<=, )[\w]+', x).group())
titleDf.head()

,Title
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr


In [13]:
'''
定义以下几种头衔类别：
Officer政府官员
Royalty王室（皇室）
Mr已婚男士
Mrs已婚妇女
Miss年轻未婚女子
Master有技能的人/教师
'''
#姓名中头衔字符串与定义头衔类别的映射关系
title_mapDict = {"Capt":       "Officer",
                 "Col":        "Officer",
                 "Major":      "Officer",
                 "Jonkheer":   "Royalty",
                 "Don":        "Royalty",
                 "Sir" :       "Royalty",
                 "Dr":         "Officer",
                 "Rev":        "Officer",
                 "the Countess":"Royalty",
                 "Dona":       "Royalty",
                 "Mme":        "Mrs",
                 "Mlle":       "Miss",
                 "Ms":         "Mrs",
                 "Mr" :        "Mr",
                 "Mrs" :       "Mrs",
                 "Miss" :      "Miss",
                 "Master" :    "Master",
                 "Lady" :      "Royalty"
                }
titleDf['Title'] = titleDf['Title'].map(title_mapDict)
#使用get_dummies进行one-hot编码
titleDf = pd.get_dummies(titleDf['Title'])
titleDf.head()

,Master,Miss,Mr,Mrs,Officer,Royalty
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,1,0,0
4,0,0,1,0,0,0


In [14]:
#提取客舱号首字母
#存放客舱号信息b
cabinDf = pd.DataFrame()
full[ 'Cabin' ] = full[ 'Cabin' ].map(lambda x: x[0])
#使用get_dummies进行one-hot编码，列名前缀是Cabin
cabinDf = pd.get_dummies( full['Cabin'] , prefix = 'Cabin' )
cabinDf.head()

,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1


In [15]:
#建立家庭人数和家庭类别
#家庭人数 = 同代直系亲属数（Parch）+ 不同代直系亲属数（SibSp）+ 乘客自己
#存放客舱号信息
familyDf = pd.DataFrame()
familyDf[ 'FamilySize' ] = full[ 'Parch' ] + full[ 'SibSp' ] + 1
'''
家庭类别：
小家庭Family_Single：家庭人数=1
中等家庭Family_Small: 2<=家庭人数<=4
大家庭Family_Large: 家庭人数>=5
'''
familyDf['Family_Single'] = familyDf['FamilySize'].map( lambda x: 1 if x == 1 else 0 )
familyDf['Family_Small']  = familyDf['FamilySize'].map( lambda x: 1 if 2 <= x <= 4 else 0 )
familyDf['Family_Large']  = familyDf['FamilySize'].map( lambda x: 1 if 5 <= x else 0 )
familyDf.head()

,FamilySize,Family_Single,Family_Small,Family_Large
0,2,0,1,0
1,2,0,1,0
2,1,1,0,0
3,2,0,1,0
4,1,1,0,0


In [16]:
full = pd.concat([full, embarkedDf, pclassDf, titleDf, cabinDf, familyDf], axis=1)
full.drop(['Embarked', 'Pclass', 'Name', 'Cabin'], axis=1, inplace=True)
print(full.shape)
full.head()

(1309, 33)


,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Embarked_C,Embarked_Q,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,FamilySize,Family_Single,Family_Small,Family_Large
0,22.0,7.2500,0,1,1,1,0.0,A/5 21171,0,0,...,0,0,0,0,0,1,2,0,1,0
1,38.0,71.2833,0,2,0,1,1.0,PC 17599,1,0,...,0,0,0,0,0,0,2,0,1,0
2,26.0,7.9250,0,3,0,0,1.0,STON/O2. 3101282,0,0,...,0,0,0,0,0,1,1,1,0,0
3,35.0,53.1000,0,4,0,1,1.0,113803,0,0,...,0,0,0,0,0,0,2,0,1,0
4,35.0,8.0500,0,5,1,0,0.0,373450,0,0,...,0,0,0,0,0,1,1,1,0,0


特征工程

In [20]:
#相关性矩阵
corrDf = full.corr()
corrDf

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Embarked_C,Embarked_Q,Embarked_S,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,FamilySize,Family_Single,Family_Small,Family_Large
Age,1.000000,0.171521,-0.130872,0.025731,0.057397,-0.190747,-0.070323,0.076179,-0.012718,-0.059153,...,0.132886,0.106600,-0.072644,-0.085977,0.032461,-0.271918,-0.196996,0.116675,-0.038189,-0.161210
Fare,0.171521,1.000000,0.221522,0.031416,-0.185484,0.160224,0.257307,0.286241,-0.130054,-0.169894,...,0.072737,0.073949,-0.037567,-0.022857,0.001179,-0.507197,0.226465,-0.274826,0.197281,0.170853
Parch,-0.130872,0.221522,1.000000,0.008942,-0.213125,0.373587,0.081629,-0.008635,-0.100943,0.071881,...,-0.027385,0.001084,0.020481,0.058325,-0.012304,-0.036806,0.792296,-0.549022,0.248532,0.624627
PassengerId,0.025731,0.031416,0.008942,1.000000,0.013406,-0.055224,-0.005007,0.048101,0.011585,-0.049836,...,0.000549,-0.008136,0.000306,-0.045949,-0.023049,0.000208,-0.031437,0.028546,0.002975,-0.063415
Sex,0.057397,-0.185484,-0.213125,0.013406,1.000000,-0.109609,-0.543351,-0.066564,-0.088651,0.115193,...,-0.057396,-0.040340,-0.006655,-0.083285,0.020558,0.137396,-0.188583,0.284537,-0.255196,-0.077748
SibSp,-0.190747,0.160224,0.373587,-0.055224,-0.109609,1.000000,-0.035322,-0.048396,-0.048678,0.073709,...,-0.015727,-0.027180,-0.008619,0.006015,-0.013247,0.009064,0.861952,-0.591077,0.253590,0.699681
Survived,-0.070323,0.257307,0.081629,-0.005007,-0.543351,-0.035322,1.000000,0.168240,0.003650,-0.149683,...,0.150716,0.145321,0.057935,0.016040,-0.026456,-0.316912,0.016639,-0.203367,0.279855,-0.125147
Embarked_C,0.076179,0.286241,-0.008635,0.048101,-0.066564,-0.048396,0.168240,1.000000,-0.164166,-0.778262,...,0.107782,0.027566,-0.020010,-0.031566,-0.014095,-0.258257,-0.036553,-0.107874,0.159594,-0.092825
Embarked_Q,-0.012718,-0.130054,-0.100943,0.011585,-0.088651,-0.048678,0.003650,-0.164166,1.000000,-0.491656,...,-0.061459,-0.042877,-0.020282,-0.019941,-0.008904,0.142369,-0.087190,0.127214,-0.122491,-0.018423
Embarked_S,-0.059153,-0.169894,0.071881,-0.049836,0.115193,0.073709,-0.149683,-0.778262,-0.491656,1.000000,...,-0.056023,0.002960,0.030575,0.040560,0.018111,0.137351,0.087771,0.014246,-0.062909,0.093671


In [21]:
corrDf['Survived'].sort_values(ascending =False)

Survived         1.000000
Mrs              0.344935
Miss             0.332795
Pclass_1         0.285904
Family_Small     0.279855
Fare             0.257307
Cabin_B          0.175095
Embarked_C       0.168240
Cabin_D          0.150716
Cabin_E          0.145321
Cabin_C          0.114652
Pclass_2         0.093349
Master           0.085221
Parch            0.081629
Cabin_F          0.057935
Cabin_A          0.022287
FamilySize       0.016639
Cabin_G          0.016040
Royalty          0.016040
Embarked_Q       0.003650
PassengerId     -0.005007
Cabin_T         -0.026456
Officer         -0.031316
SibSp           -0.035322
Age             -0.070323
Family_Large    -0.125147
Embarked_S      -0.149683
Family_Single   -0.203367
Cabin_U         -0.316912
Pclass_3        -0.322308
Sex             -0.543351
Mr              -0.549199
Name: Survived, dtype: float64

从表中可以发现与生存情况线性相关的特征。

In [22]:
#特征选择
full_X = pd.concat([titleDf,#头衔
                    pclassDf,#客舱等级
                    familyDf,#家庭大小
                    full['Fare'],#船票价格
                    cabinDf,#船舱号
                    embarkedDf,#登船港口
                    full['Sex']#性别
                   ],
                   axis=1)
full_X.head()

,Master,Miss,Mr,Mrs,Officer,Royalty,Pclass_1,Pclass_2,Pclass_3,FamilySize,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S,Sex
0,0,0,1,0,0,0,0,0,1,2,...,0,0,0,0,0,1,0,0,1,1
1,0,0,0,1,0,0,1,0,0,2,...,0,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,1,0
3,0,0,0,1,0,0,1,0,0,2,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,1,1


4.构建模型

In [23]:
#原始数据集有891行
sourceRow=891
#原始数据集：特征
source_X = full_X.loc[0:sourceRow-1,:]
#原始数据集：标签
source_y = full.loc[0:sourceRow-1,'Survived']
#预测数据集：特征
pred_X = full_X.loc[sourceRow:,:]

In [24]:
#建立模型用的训练数据集和测试数据集
from sklearn.model_selection import train_test_split 

train_X, test_X, train_y, test_y = train_test_split(source_X , source_y, train_size=0.8, test_size=0.2)
#输出数据集大小
print ('原始数据集特征：',source_X.shape, 
       '训练数据集特征：',train_X.shape ,
      '测试数据集特征：',test_X.shape)
print ('原始数据集标签：',source_y.shape, 
       '训练数据集标签：',train_y.shape ,
      '测试数据集标签：',test_y.shape)

原始数据集特征： (891, 27) 训练数据集特征： (712, 27) 测试数据集特征： (179, 27)
原始数据集标签： (891,) 训练数据集标签： (712,) 测试数据集标签： (179,)


In [26]:
#第1步：导入算法
from sklearn.linear_model import LogisticRegression
#第2步：创建模型：逻辑回归（logisic regression）
model = LogisticRegression()

In [ ]:
#随机森林Random Forests Model
#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(n_estimators=100)

In [ ]:
#支持向量机Support Vector Machines
#from sklearn.svm import SVC, LinearSVC
#model = SVC()

In [ ]:
#Gradient Boosting Classifier
#from sklearn.ensemble import GradientBoostingClassifier
#model = GradientBoostingClassifier()

In [ ]:
#K-nearest neighbors
#from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
# Gaussian Naive Bayes
#from sklearn.naive_bayes import GaussianNB
#model = GaussianNB()

In [27]:
#第3步：训练模型
model.fit(train_X , train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

5.模型评估

In [28]:
model.score(test_X , test_y )

0.8324022346368715

6.方案实施

In [30]:
#使用机器学习模型，对预测数据集中的生存情况进行预测
pred_Y = model.predict(pred_X)
#生成的预测值是浮点数,转换为整型
pred_Y=pred_Y.astype(int)
#乘客id
passenger_id = full.loc[sourceRow:,'PassengerId']
#数据框：乘客id，预测生存情况的值
predDf = pd.DataFrame({'PassengerId': passenger_id,
                       'Survived': pred_Y})
print(predDf.shape)
predDf.head()

(418, 2)


,PassengerId,Survived
891,892,0
892,893,1
893,894,0
894,895,0
895,896,1


In [ ]:
#保存结果
predDf.to_csv('titanic_pred.csv', index = False)